In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [ ]:
!wget --show-progress --continue -O /content/shakespeare.txt http://www.gutenberg.org/files/100/100-0.txt

--2020-03-26 18:48:11--  http://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5777367 (5.5M) [text/plain]
Saving to: ‘/content/shakespeare.txt’

/content/shakespear 100%[===================>]   5.51M  5.37MB/s    in 1.0s    

2020-03-26 18:48:17 (5.37 MB/s) - ‘/content/shakespeare.txt’ saved [5777367/5777367]



In [ ]:
import numpy as np
import tensorflow as tf
import os

In [ ]:
TEST_FILE = '/content/test.txt'
SHAKESPEARE_TXT = '/content/shakespeare.txt'

In [ ]:
def transfor(txt):
  return np.asarray([ord(c) for c in txt if ord(c) < 255], dtype=np.int32)

In [ ]:
with tf.io.gfile.GFile(TEST_FILE, 'r') as f:
  txt = f.read()
transfor(txt)

array([ 73, 116,  32, 105, 115,  32,  97,  32,  98, 101,  97, 117, 116,
       105, 102, 117, 108,  32, 100,  97, 121,  44,  32, 114, 105, 103,
       104, 116,  63,  32,  10,  76, 101, 116,  39, 115,  32, 103, 111,
        32, 111, 117, 116,  32,  97, 110, 100,  32, 104,  97, 118, 101,
        32, 102, 117, 110,  33], dtype=int32)

In [ ]:
#each input_size(sentence 字母数量) = seq_len, 
#每个batch 中 有 batch_size个 (input->target)的sets
@tf.function
def input_fn(seq_len=10, batch_size=2):
  with tf.io.gfile.GFile(TEST_FILE, 'r') as f:
    txt = f.read()
  
  source = tf.constant(transfor(txt), dtype=tf.int32)
  #把tensor类型转化为dataset类型
  ds = tf.data.Dataset.from_tensor_slices(source)
  #Split the txt into sentenes, each sentences have seq_len+1 characters
  ds = ds.batch(seq_len+1, True)
  return ds


In [ ]:
#make every sentences into:
#   input sentence(first character to last-1 character)
#   target_sentence(second char to last character)
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

BUFFER_SIZE = 10000

In [ ]:
ds = input_fn()

In [ ]:
list(ds.as_numpy_iterator())

[array([ 73, 116,  32, 105, 115,  32,  97,  32,  98, 101,  97], dtype=int32),
 array([117, 116, 105, 102, 117, 108,  32, 100,  97, 121,  44], dtype=int32),
 array([ 32, 114, 105, 103, 104, 116,  63,  32,  10,  76, 101], dtype=int32),
 array([116,  39, 115,  32, 103, 111,  32, 111, 117, 116,  32], dtype=int32),
 array([ 97, 110, 100,  32, 104,  97, 118, 101,  32, 102, 117], dtype=int32)]

In [ ]:
ds

<_VariantDataset shapes: (11,), types: tf.int32>

In [ ]:
ds = ds.map(split_input_target)

In [ ]:
ds

<MapDataset shapes: ((10,), (10,)), types: (tf.int32, tf.int32)>

In [ ]:
list(ds.as_numpy_iterator())

[(array([ 73, 116,  32, 105, 115,  32,  97,  32,  98, 101], dtype=int32),
  array([116,  32, 105, 115,  32,  97,  32,  98, 101,  97], dtype=int32)),
 (array([117, 116, 105, 102, 117, 108,  32, 100,  97, 121], dtype=int32),
  array([116, 105, 102, 117, 108,  32, 100,  97, 121,  44], dtype=int32)),
 (array([ 32, 114, 105, 103, 104, 116,  63,  32,  10,  76], dtype=int32),
  array([114, 105, 103, 104, 116,  63,  32,  10,  76, 101], dtype=int32)),
 (array([116,  39, 115,  32, 103, 111,  32, 111, 117, 116], dtype=int32),
  array([ 39, 115,  32, 103, 111,  32, 111, 117, 116,  32], dtype=int32)),
 (array([ 97, 110, 100,  32, 104,  97, 118, 101,  32, 102], dtype=int32),
  array([110, 100,  32, 104,  97, 118, 101,  32, 102, 117], dtype=int32))]

In [ ]:
#Shuffle the (input->target) sets
ds = ds.shuffle(10000)

In [ ]:
list(ds.as_numpy_iterator())

[(array([ 73, 116,  32, 105, 115,  32,  97,  32,  98, 101], dtype=int32),
  array([116,  32, 105, 115,  32,  97,  32,  98, 101,  97], dtype=int32)),
 (array([ 97, 110, 100,  32, 104,  97, 118, 101,  32, 102], dtype=int32),
  array([110, 100,  32, 104,  97, 118, 101,  32, 102, 117], dtype=int32)),
 (array([ 32, 114, 105, 103, 104, 116,  63,  32,  10,  76], dtype=int32),
  array([114, 105, 103, 104, 116,  63,  32,  10,  76, 101], dtype=int32)),
 (array([116,  39, 115,  32, 103, 111,  32, 111, 117, 116], dtype=int32),
  array([ 39, 115,  32, 103, 111,  32, 111, 117, 116,  32], dtype=int32)),
 (array([117, 116, 105, 102, 117, 108,  32, 100,  97, 121], dtype=int32),
  array([116, 105, 102, 117, 108,  32, 100,  97, 121,  44], dtype=int32))]

In [ ]:
#Split those (input->target) sets into batches, each batch has Batch_size 个 (input->target) sets
ds = ds.batch(2, True)

In [ ]:
list(ds.as_numpy_iterator())

[(array([[117, 116, 105, 102, 117, 108,  32, 100,  97, 121],
         [116,  39, 115,  32, 103, 111,  32, 111, 117, 116]], dtype=int32),
  array([[116, 105, 102, 117, 108,  32, 100,  97, 121,  44],
         [ 39, 115,  32, 103, 111,  32, 111, 117, 116,  32]], dtype=int32)),
 (array([[ 97, 110, 100,  32, 104,  97, 118, 101,  32, 102],
         [ 32, 114, 105, 103, 104, 116,  63,  32,  10,  76]], dtype=int32),
  array([[110, 100,  32, 104,  97, 118, 101,  32, 102, 117],
         [114, 105, 103, 104, 116,  63,  32,  10,  76, 101]], dtype=int32))]

In [ ]:
ds

<BatchDataset shapes: ((2, 10), (2, 10)), types: (tf.int32, tf.int32)>

In [ ]:
ds = ds.repeat()

In [ ]:
ds

<RepeatDataset shapes: ((2, 10), (2, 10)), types: (tf.int32, tf.int32)>

总结上面的代码

In [ ]:
#总结以上
def input_fn(seq_len=100, batch_size=1024):
  with tf.io.gfile.GFile(SHAKESPEARE_TXT,'r') as f:
    txt = f.read()
  
  source = tf.constant(transfor(txt), dtype=tf.int32)

  ds = tf.data.Dataset.from_tensor_slices(source)
  ds = ds.batch(seq_len+1, drop_remainder=True)

  def split_input_target(chunk):
    input_txt = chunk[:-1]
    output_txt = chunk[1:]
    return input_txt, output_txt

  SHUFFLE_BUFFER_SIZE = 1000
  ds = ds.map(split_input_target)
  ds = ds.shuffle(SHUFFLE_BUFFER_SIZE)
  ds = ds.batch(batch_size, drop_remainder=True)

  return ds.repeat()

In [ ]:
EMBEDDING_DIM = 512
#规定input shape is (Batch_size, Seq_len(每句话有几个字母))
def lstm_model(seq_len=10, batch_size=None, stateful=True):
  source = tf.keras.Input(name='seed', shape=(seq_len,),batch_size=batch_size, dtype=tf.int32)

  return source

In [ ]:
source = lstm_model(10, None, False)
source

<tf.Tensor 'seed:0' shape=(None, 10) dtype=int32>

In [ ]:
#Embedding Layer(input_dim= Vocabulary Size, output_dim = Dimension of output, input_len=Sequence_len(一句话有几个字母))
#input_len has to be set when connect with Flatten or Dense layer
#Input_shape => source (Batch_size, Sequence_len)
#把一个字母变成一个shape为(256,)的数组
embedding = tf.keras.layers.Embedding(input_dim=256, output_dim=EMBEDDING_DIM)(source)
embedding
#output_shape => (Batch_size, Sequence_len, output_dim)

<tf.Tensor 'embedding_1/Identity:0' shape=(None, 10, 512) dtype=float32>

In [ ]:
#input_shape (Batch_size=None, TimeSteps/input_len/Sequence_len=10, input_dimension=512)
lstm_1 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=False, return_sequences=True)(embedding)
lstm_1
#output_shape (Batch_size=None, Timesteps=10, out_dim=512)

<tf.Tensor 'lstm_2/Identity:0' shape=(None, 10, 512) dtype=float32>

In [ ]:
#input_shape (Batch_size=None, TimeSteps/input_len/Sequence_len=10, input_dimension=512)
lstm_2 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=False, return_sequences=True)(lstm_1)
lstm_2
#output_shape (Batch_size=None, Timesteps=10, out_dim=512)

<tf.Tensor 'lstm_3/Identity:0' shape=(None, 10, 512) dtype=float32>

In [ ]:
predicted_char = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256,activation='softmax'))(lstm_2)
predicted_char

<tf.Tensor 'time_distributed_1/Identity:0' shape=(None, 10, 256) dtype=float32>

In [ ]:
#Model代码总结

In [ ]:
model = tf.keras.Model(inputs=[source], outputs=[predicted_char])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
seed (InputLayer)            [(None, 10)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 512)           131072    
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 512)           2099200   
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 512)           2099200   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 256)           131328    
Total params: 4,460,800
Trainable params: 4,460,800
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Summary above
EMBEDDING_DIM = 512

def lstm_model(seq_len=100, batch_size=None, stateful=True):
  """Language model: predict the next word given the current word."""
  source = tf.keras.Input(
      name='seed', shape=(seq_len,), batch_size=batch_size, dtype=tf.int32)

  embedding = tf.keras.layers.Embedding(input_dim=256, output_dim=EMBEDDING_DIM)(source)
  lstm_1 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(embedding)
  lstm_2 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(lstm_1)
  predicted_char = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='softmax'))(lstm_2)
  return tf.keras.Model(inputs=[source], outputs=[predicted_char])

In [ ]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

In [ ]:
tf.keras.backend.clear_session()
resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

with strategy.scope():
  model = lstm_model(seq_len=100, stateful=False)
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01),
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy']
        )

INFO:tensorflow:Initializing the TPU system: 10.9.96.90:8470
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.9.96.90:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 12779657112255270877)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4157330350771135454)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 1717799881030086658)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 15213632275795428761)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:T

In [ ]:
model.fit(
    input_fn(), #ds中每条都是（input->target)的set
    steps_per_epoch=100,
    epochs=10
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
100/100 [==============================] - 13s 127ms/step - loss: 3.4938 - sparse_categorical_accuracy: 0.1590
Epoch 2/10
100/100 [==============================] - 9s 94ms/step - loss: 3.0443 - sparse_categorical_accuracy: 0.1968
Epoch 3/10
100/100 [==============================] - 9s 94ms/step - loss: 2.7753 - sparse_categorical_accuracy: 0.2635
Epoch 4/10
100/100 [==============================] - 9s 95ms/step - loss: 2.2097 - sparse_categorical_accuracy: 0.3714
Epoch 5/10
100/100 [==============================] - 10s 95ms/step - loss: 1.7072 - sparse_categorical_accuracy: 0.4936
Epoch 6/10
100/100 [==============================] - 10s 96ms/step - loss: 1.4450 - sparse_categorical_accuracy: 0.5635
Epoch 7/10
100/100 [==============================] - 10s 96ms/step - loss: 1.3398 - sparse_categorical_accuracy: 0.5906
Epoch 8/10
100/100 [==============================] - 10s 98m

In [ ]:
model.save_weights('/tmp/bard.h5', overwrite=True)

In [ ]:
#给要用来预测的话做 5 组不同的预测，即5次预测
BATCH_SIZE = 5
#每组预测250个字母
PREDICT_LEN = 250

In [ ]:
#set stateful=True so model's state is kept between batches. 
#If stateful=False, the model state is reset between each batch
#and the model will only be able to use the information from the current batch (a single character) 
#to make a prediction.

#当stateful为True时，必须要set Batch_size
#因为我们一个一个字母的feed给model 所以把seq_len设为1
prediction_model = lstm_model(seq_len=1, batch_size=BATCH_SIZE, stateful=True)
prediction_model.load_weights('/tmp/bard.h5')

In [ ]:
# We seed the model with our initial string, copied BATCH_SIZE times
seed_txt = 'Looks it not like the king?  Verily, we must go! '
seed = transfor(seed_txt)
seed

array([ 76, 111, 111, 107, 115,  32, 105, 116,  32, 110, 111, 116,  32,
       108, 105, 107, 101,  32, 116, 104, 101,  32, 107, 105, 110, 103,
        63,  32,  32,  86, 101, 114, 105, 108, 121,  44,  32, 119, 101,
        32, 109, 117, 115, 116,  32, 103, 111,  33,  32], dtype=int32)

In [ ]:
#给要用来预测的话加上一个维度（之后好整段repeat5次）
seed = np.expand_dims(seed, 0)
seed

array([[ 76, 111, 111, 107, 115,  32, 105, 116,  32, 110, 111, 116,  32,
        108, 105, 107, 101,  32, 116, 104, 101,  32, 107, 105, 110, 103,
         63,  32,  32,  86, 101, 114, 105, 108, 121,  44,  32, 119, 101,
         32, 109, 117, 115, 116,  32, 103, 111,  33,  32]], dtype=int32)

In [ ]:
#seed = np.repeat(seed, 2, axis=1)

In [ ]:
#seed

array([[ 76,  76, 111, 111, 111, 111, 107, 107, 115, 115,  32,  32, 105,
        105, 116, 116,  32,  32, 110, 110, 111, 111, 116, 116,  32,  32,
        108, 108, 105, 105, 107, 107, 101, 101,  32,  32, 116, 116, 104,
        104, 101, 101,  32,  32, 107, 107, 105, 105, 110, 110, 103, 103,
         63,  63,  32,  32,  32,  32,  86,  86, 101, 101, 114, 114, 105,
        105, 108, 108, 121, 121,  44,  44,  32,  32, 119, 119, 101, 101,
         32,  32, 109, 109, 117, 117, 115, 115, 116, 116,  32,  32, 103,
        103, 111, 111,  33,  33,  32,  32]], dtype=int32)

In [ ]:
#将整句话复制5遍
seed = np.repeat(seed, BATCH_SIZE, axis=0)
seed

array([[ 76, 111, 111, 107, 115,  32, 105, 116,  32, 110, 111, 116,  32,
        108, 105, 107, 101,  32, 116, 104, 101,  32, 107, 105, 110, 103,
         63,  32,  32,  86, 101, 114, 105, 108, 121,  44,  32, 119, 101,
         32, 109, 117, 115, 116,  32, 103, 111,  33,  32],
       [ 76, 111, 111, 107, 115,  32, 105, 116,  32, 110, 111, 116,  32,
        108, 105, 107, 101,  32, 116, 104, 101,  32, 107, 105, 110, 103,
         63,  32,  32,  86, 101, 114, 105, 108, 121,  44,  32, 119, 101,
         32, 109, 117, 115, 116,  32, 103, 111,  33,  32],
       [ 76, 111, 111, 107, 115,  32, 105, 116,  32, 110, 111, 116,  32,
        108, 105, 107, 101,  32, 116, 104, 101,  32, 107, 105, 110, 103,
         63,  32,  32,  86, 101, 114, 105, 108, 121,  44,  32, 119, 101,
         32, 109, 117, 115, 116,  32, 103, 111,  33,  32],
       [ 76, 111, 111, 107, 115,  32, 105, 116,  32, 110, 111, 116,  32,
        108, 105, 107, 101,  32, 116, 104, 101,  32, 107, 105, 110, 103,
         63,  32,  3

---------------------------
testing

In [ ]:
len(seed_txt)

49

In [ ]:
preds = [seed[:, -1:]]
preds

[array([[32],
        [32],
        [32],
        [32],
        [32]], dtype=int32)]

In [ ]:
preds = seed[0, -1:]
preds

array([32], dtype=int32)

In [ ]:
last_word = predictions[-1]
last_word

array([[32],
       [32],
       [32],
       [32],
       [32]], dtype=int32)

-------------------------------------
test end

In [ ]:
prediction_model.reset_states()

In [ ]:
#Run the seed forward to prime the state of model
for i in range(len(seed_txt)-1):
  print(seed[:, i:i+1])
  prediction_model.predict(seed[:, i:i+1])

[[76]
 [76]
 [76]
 [76]
 [76]]
ERROR! Session/line number was not unique in database. History logging moved to new session 59
[[111]
 [111]
 [111]
 [111]
 [111]]
[[111]
 [111]
 [111]
 [111]
 [111]]
[[107]
 [107]
 [107]
 [107]
 [107]]
[[115]
 [115]
 [115]
 [115]
 [115]]
[[32]
 [32]
 [32]
 [32]
 [32]]
[[105]
 [105]
 [105]
 [105]
 [105]]
[[116]
 [116]
 [116]
 [116]
 [116]]
[[32]
 [32]
 [32]
 [32]
 [32]]
[[110]
 [110]
 [110]
 [110]
 [110]]
[[111]
 [111]
 [111]
 [111]
 [111]]
[[116]
 [116]
 [116]
 [116]
 [116]]
[[32]
 [32]
 [32]
 [32]
 [32]]
[[108]
 [108]
 [108]
 [108]
 [108]]
[[105]
 [105]
 [105]
 [105]
 [105]]
[[107]
 [107]
 [107]
 [107]
 [107]]
[[101]
 [101]
 [101]
 [101]
 [101]]
[[32]
 [32]
 [32]
 [32]
 [32]]
[[116]
 [116]
 [116]
 [116]
 [116]]
[[104]
 [104]
 [104]
 [104]
 [104]]
[[101]
 [101]
 [101]
 [101]
 [101]]
[[32]
 [32]
 [32]
 [32]
 [32]]
[[107]
 [107]
 [107]
 [107]
 [107]]
[[105]
 [105]
 [105]
 [105]
 [105]]
[[110]
 [110]
 [110]
 [110]
 [110]]
[[103]
 [103]
 [103]
 [103]
 [103]]

In [ ]:
# Now we can accumulate predictions!
predictions = [seed[:, -1:]]

[array([[32],
        [32],
        [32],
        [32],
        [32]], dtype=int32)]

In [ ]:
test = np.array([[[2,3]],[[2,3]],[[2,3]]])
test.shape

(3, 1, 2)

________________________
解释用

In [ ]:
for i in range(1):
  #batch_size 设为 5,即输入进去每组5个，每个为当前predictions中的最后1个字母
  #即shape为（batch_size=5, timeSteps/seq_len=1, input_dim/Vocabularies=256)
  last_word = predictions[-1]
  #输出的结果的shape为（batch_size=5, timeSteps/seq_len=1, input_dim/Vocabularies=256)
  next_probits = prediction_model.predict(x=last_word)
  print(next_probits)
  #batch_size的5个都保留，因为每个seq_len/timeSteps中只有1个字母，即为第0项
  #每个seq_len有256个维度（即字典内共256个字母）
  next_probits = next_probits[:, 0, :]
  print(next_probits)

  #用预测出的概率，算出batch中的5个字母分别预测到的是什么字母
  next_idx = [         #a=256代表np.arange(256)->[0,1,2,3,...,255]
      np.random.choice(a=256, p=next_probits[i]) #p表示数组a中对应每个位置上可能被选中的概率
      for i in range(BATCH_SIZE)
  ]
  predictions.append(np.asarray(next_idx, dtype=np.int32))
  print(predictions)
  

[[[3.3767023e-10 3.0895714e-10 2.5281263e-10 ... 4.2209461e-10
   2.3955368e-10 3.7252418e-10]]

 [[3.3767023e-10 3.0895714e-10 2.5281263e-10 ... 4.2209461e-10
   2.3955368e-10 3.7252418e-10]]

 [[3.3767023e-10 3.0895714e-10 2.5281263e-10 ... 4.2209461e-10
   2.3955368e-10 3.7252418e-10]]

 [[3.3767023e-10 3.0895714e-10 2.5281263e-10 ... 4.2209461e-10
   2.3955368e-10 3.7252418e-10]]

 [[3.3767023e-10 3.0895714e-10 2.5281263e-10 ... 4.2209461e-10
   2.3955368e-10 3.7252418e-10]]]
[[3.3767023e-10 3.0895714e-10 2.5281263e-10 ... 4.2209461e-10
  2.3955368e-10 3.7252418e-10]
 [3.3767023e-10 3.0895714e-10 2.5281263e-10 ... 4.2209461e-10
  2.3955368e-10 3.7252418e-10]
 [3.3767023e-10 3.0895714e-10 2.5281263e-10 ... 4.2209461e-10
  2.3955368e-10 3.7252418e-10]
 [3.3767023e-10 3.0895714e-10 2.5281263e-10 ... 4.2209461e-10
  2.3955368e-10 3.7252418e-10]
 [3.3767023e-10 3.0895714e-10 2.5281263e-10 ... 4.2209461e-10
  2.3955368e-10 3.7252418e-10]]
[array([[32],
       [32],
       [32],
       [3

______________
解释结束

In [ ]:
for i in range(PREDICT_LEN):
  #batch_size 设为 5,即输入进去每组5个，每个为当前predictions中的最后1个字母
  #即shape为（batch_size=5, timeSteps/seq_len=1, input_dim/Vocabularies=256)
  last_word = predictions[-1]
  #输出的结果的shape为（batch_size=5, timeSteps/seq_len=1, input_dim/Vocabularies=256)
  next_probits = prediction_model.predict(x=last_word)
  #batch_size的5个都保留，因为每个seq_len/timeSteps中只有1个字母，即为第0项
  #每个seq_len有256个维度（即字典内共256个字母）
  next_probits = next_probits[:, 0, :]

  #用预测出的概率，算出batch中的5个字母分别预测到的是什么字母
  next_idx = [         #a=256代表np.arange(256)->[0,1,2,3,...,255]
      np.random.choice(a=256, p=next_probits[i]) #p表示数组a中对应每个位置上可能被选中的概率
      for i in range(BATCH_SIZE)
  ]
  predictions.append(np.asarray(next_idx, dtype=np.int32))  

In [ ]:
print(predictions[:3])
#predictions 中 有PREDICT_LEN=256个array，对应往下预测256个字母
#每个array中有batch_size=5个字母，对应5组不同的预测结果


[array([[32],
       [32],
       [32],
       [32],
       [32]], dtype=int32), array([32, 32, 32, 32, 32], dtype=int32), array([32, 32, 32, 32, 32], dtype=int32)]


In [ ]:
for i in range(BATCH_SIZE):
  print('Prediction %d\n\n' % i)
  #将predictions中256个字母组合起来
  p = [predictions[j][i] for j in range(PREDICT_LEN)]
  #将unicode转换为char
  generate = ''.join([chr(c) for c in p])
  print(generate)
  print('---------------------------')
  #生成的字母必须为PREDICT_LEN个
  assert len(generate) == PREDICT_LEN

Prediction 0


                             [Tragsting i th arm upon the patland.]


                   SCENE IV. Padua.


           DOLABELLA, Julia but NURVINGMAN a bapher of OLIFF]

That I said, a modesty acquaintance in a thought upon my particular for
---------------------------
Prediction 1


                                Exeunt

          336

   And here Priam and 'These arratial come
Most, an accounted my father, but hone
Most immortal furwearing think our 
To practice the scald of Hamlet at a jest, danger. A better than us.

---------------------------
Prediction 2


                    ordement torture
    The full lamp but to be
    That the crow-trees a brace.
  VILUMNIA. Ay, for thy sisters, but revelled-time were heaven; never that he is; what
    shrubs to hear at home.
    As whom what I do bed it opp
---------------------------
Prediction 3


                   13290


An if thy villaried dishonour hath call'd thy praise,
Why do the sway to pry our power,
Horatio i